In [1]:

from cmath import nan
import json
import pandas as pd
import formula1_scrape
import numpy as np

f = open('raw_data/driver_data.json')
data = json.load(f)
df = pd.DataFrame(data)

#fix other data types
df["First Race"] = df["First Race"].astype(str)
#Consistency for Nationalities
df["Nationality"] = df["Nationality"].str.title()

#Separate circuits and years for first race and last race

first_race_data = df["First Race"].str.rsplit(n=1, expand=True)
last_race_data = df["Last Race"].str.rsplit(n=1, expand=True)

df["Debut Year"] = first_race_data[1]
df["Debut Circuit"] = first_race_data[0]
df["Debut Circuit"] = df["Debut Circuit"].astype(str)
df["Final Year"] = last_race_data[1]
df["Final Circuit"] = last_race_data[0]
df["Final Circuit"] = df["Final Circuit"].astype(str)


#Separate best position and year
df["Best championship position"] = df["Best championship position"].astype(str)
best_champseason_place = df["Best championship position"].str.split(n=1, expand=True)
df["Best championship position"] = best_champseason_place[0]
best_champseason_year = best_champseason_place[1].str.split(n=2, expand=True)
df["Best championship year"] = best_champseason_year[2]
df["Best championship position"] = df["Best championship position"].astype(str)
df["Best championship year"] = df["Best championship year"].astype(str)
df.loc[df["Best championship position"] == "World", "Best championship position"] = "1st"

    
#Convert dates/times
df["Date of birth"] = pd.to_datetime(df["Date of birth"], infer_datetime_format=True, errors = 'coerce')
df["Date of death"] = pd.to_datetime(df["Date of death"], infer_datetime_format=True, errors = 'coerce')

#fix other data types
df["Driver First Name"] = df["Driver First Name"].astype(str)
df["Driver Second Name"] = df["Driver Second Name"].astype(str)
df["Nationality"] = df["Nationality"].astype(str)
df["Hometown"] = df["Hometown"].astype(str)
df["Driver Second Name"] = df["Driver Second Name"].astype(str)
df["Points"] = df["Points"].astype(float)


In [2]:
cols = df.columns.to_list()
cols = cols[0:8]+[cols[-7]]+cols[-5:]+cols[8:-5]
df = df.drop("First Race", axis=1)
df = df.drop("Last Race", axis=1)
df = df.drop("Year active", axis=1)
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')


print(df.iloc[12,2])
clean_driver_data = df



Alan


In [4]:

df = df.replace('-',np.nan)
df = df.replace('', np.nan)
np.where(df.applymap(lambda x: x == ''))


(array([], dtype=int64), array([], dtype=int64))

In [13]:
print(df.dtypes)

id                                      object
driver_number                            int64
driver_first_name                       object
driver_second_name                      object
nationality                             object
hometown                                object
date_of_birth                   datetime64[ns]
date_of_death                   datetime64[ns]
grands_prix_entered                      int64
teams                                   object
seasons                                 object
circuits                                object
best_race_starting_position             object
best_race_finishing_position            object
victories                                int64
podiums                                  int64
pole_positions                           int64
first_row_start                          int64
fastest_lap                              int64
points_finishes                          int64
points                                 float64
best_champion

In [6]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import inspect
from sqlalchemy import exists
from sqlalchemy import text
import sqlalchemy

DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
HOST = 'formula1.c0ptp1rfwhvx.eu-west-2.rds.amazonaws.com'
USER = 'postgres'
PASSWORD = 'T00narmyf1s'
DATABASE = 'Formula1'
PORT = 5432

engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
engine.connect()

inspector = inspect(engine)
inspector.get_table_names()



whoop


In [5]:
clean_driver_data.to_sql('Drivers', engine, if_exists='replace')

852

In [27]:
pkey = engine.execute(text('SELECT indexname, indexdef FROM pg_indexes WHERE tablename = \'"Drivers"\';'))
print(pkey)

In [30]:
idx = clean_driver_data[clean_driver_data['Driver Second Name'] == 'Senna'].index.values[0]
clean_driver_data.iloc[idx:].to_sql('Drivers', engine, if_exists='append', index=False)

789

In [8]:
f = open('teams_data.json')
data = json.load(f)
df = pd.DataFrame(data)
print(df["First Race"])


#Separate circuits and years for first race and last race

first_race_data = df["First Race"].str.rsplit(n=1, expand=True)
last_race_data = df["Last Race"].str.rsplit(n=1, expand=True)

df["Debut Year"] = first_race_data[1]
df["Debut Circuit"] = first_race_data[0]
df["Debut Circuit"] = df["Debut Circuit"].astype(str)
df["Final Year"] = last_race_data[1]
df["Final Circuit"] = last_race_data[0]
df["Final Circuit"] = df["Final Circuit"].astype(str)

#Separate best position and year
df["Best championship position (constructor)"] = df["Best championship position (constructor)"].astype(str)
best_champseason_place = df["Best championship position (constructor)"].str.split(n=1, expand=True)
print(best_champseason_place)
df["Best championship position"] = best_champseason_place[0]
df["Best championship year"] = best_champseason_place[1]
df["Best championship position"] = df["Best championship position"].astype(str)
df["Best championship year"] = df["Best championship year"].astype(str)
df.loc[df["Best championship position"] == "World", "Best championship position"] = "1st"
best_champseason_year = best_champseason_place[1].str.split(n=2, expand=True)
df["Best championship year"] = best_champseason_year[1]

df["Best championship position (driver)"] = df["Best championship position (driver)"].astype(str)
best_champdriverseason_place = df["Best championship position (driver)"].str.split(n=2, expand=True)
print(best_champdriverseason_place)
df["Best championship position (driver)"] = best_champdriverseason_place[0]
df["Best championship year (driver)"] = best_champdriverseason_place[1]
df["Best championship driver"] = best_champdriverseason_place[2]

0        Anderstorp 1974
1           Aintree 1959
2                    NaN
3       Nurburgring 1952
4       Silverstone 1954
             ...        
462    Buenos Aires 1977
463     Nurburgring 1953
464     Monte Carlo 1960
465      Bremgarten 1951
466         Estoril 1985
Name: First Race, Length: 467, dtype: object
        0     1
0    None  None
1    None  None
2     nan  None
3    None  None
4    None  None
..    ...   ...
462     4  1977
463  None  None
464  None  None
465  None  None
466    10  1987

[467 rows x 2 columns]
       0     1                      2
0    nan  None                   None
1    nan  None                   None
2    nan  None                   None
3    nan  None                   None
4     14  1954          Stirling Moss
..   ...   ...                    ...
462    2  1977         Jody Scheckter
463    5  1962             Dan Gurney
464    5  1960              Phil Hill
465   15  1951  Yves Giraud-Cabantous
466   18  1987         Martin Brundle

[467 ro

In [9]:
cols = df.columns.to_list()
df = df.drop("First Race", axis=1)
df = df.drop("Last Race", axis=1)
df = df.drop("Best championship position (constructor)", axis=1)
clean_team_data = df
clean_team_data.to_sql('Teams', engine, if_exists='append')

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Team number" of relation "Teams" does not exist
LINE 1: INSERT INTO "Teams" (index, "ID", "Team number", "Team Name"...
                                          ^

[SQL: INSERT INTO "Teams" (index, "ID", "Team number", "Team Name", "Name", "Year active", "Best race starting position", "Best race finishing position", "Drivers entrance", "Race starts", "Seasons", "Circuits", "Drivers", "First row starts", "Two drivers on first row", "Two drivers finish 1 and 2", "All drivers on podium", "All drivers in points", "Victories", "Podiums", "Pole pos.", "Fastest lap", "In points", "Points", "Best championship position (driver)", "Debut Year", "Debut Circuit", "Final Year", "Final Circuit", "Best championship position", "Best championship year", "Best championship year (driver)", "Best championship driver") VALUES (%(index)s, %(ID)s, %(Team number)s, %(Team Name)s, %(Name)s, %(Year active)s, %(Best race starting position)s, %(Best rac

In [10]:
f = open('champs_data.json')
data = json.load(f)
df = pd.DataFrame(data)
df.to_sql('Champions', engine, if_exists='append')

72

In [12]:
f = open('circuit_data.json')
data = json.load(f)
df = pd.DataFrame(data)
df.to_sql('Circuits', engine, if_exists='append')

77

In [31]:
new_clean_driver_data = clean_driver_data.copy()
old_clean_driver_data = pd.read_sql_query('''SELECT * from "Drivers"''', con =engine)

In [32]:
merged_dfs = pd.concat([old_clean_driver_data, new_clean_driver_data]).duplicated()

In [35]:
new_values = merged_dfs[~merged_dfs.duplicated(keep=False)]
new_values.to_sql('Drivers', engine, if_exists='append', index=False)

0